In [1]:
import string
import numbers
import numpy as np
import pandas as pd
import cv2 as cv
import pprint
import scipy
import scipy.special
import scipy.spatial
import scipy.stats
import scipy.stats.qmc
import scipy.ndimage
import shapely
import shapely.geometry
import matplotlib
import matplotlib.patches
import matplotlib.patches as patches
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
import matplotlib.collections as mc
from matplotlib.path import Path
from tabulate import tabulate
import networkx as nx
import math as m
import torch

import utility as util
pp = pprint.PrettyPrinter(indent=4)

In [2]:
# Stochastic MDP homework
P = np.array([
    [0.2, 0.64, 0.16, 0],
    [0.5, 0,    0.5,  0],
    [0,   0,    0.5,  0.5],
    [0,   1,    0,    0]
])
# Expected immediate reward for next transition
R = np.array([1.96, 0.5, 2, 1])
# Value function V(s) = E_{a~\pi(s)}[r(s,a) + \gamma \sum_{s'} P(s' | s, a) V(s')]
# Expression for value function V = R + \gamma P V => (I - \gamma P) V = R
# A = I - \gamma P; \gamma = 0.8
A = np.identity(4) - 0.8*P
np.linalg.solve(A, R)
# gives np.array([7.37870117, 6.42137348, 7.42473252, 6.13709878])
V = np.array([7.37870117, 6.42137348, 7.42473252, 6.13709878])

In [82]:
# Neural Network homework
# Based on:
# https://mattmazur.com/2015/03/17/a-step-by-step-backpropagation-example/
sigmoid = scipy.special.expit
def dsigmoid(s):
    return sigmoid(s)*(1 - sigmoid(s))

# constants
alpha = 0.5
x = np.array([0.3, 0.5])
W1 = np.array([[0.5, 0.55], [0.6, 0.65]])
(w1, w2), (w3, w4) = W1
b1 = np.array([0.35, 0.35])
W2 = np.array([[0.7, 0.75], [0.8, 0.85]])
(w5, w6), (w7, w8) = W2
b2 = np.array([0.6, 0.6])
y = np.array([0.01, 0.99])
# predictions
yhat = sigmoid(W2 @ sigmoid(W1 @ x + b1) + b2)
# error
Etotal = sum(0.5*(y - yhat)**2)

# partial results
h = sigmoid(W1 @ x + b1)
d1d2 = W1 @ x + b1
d3d4 = W2 @ sigmoid(W1 @ x + b1) + b2
c5c6 = y - yhat

# \partial Etotal / \partial w5
dLdw5 = -1 * h[0] * dsigmoid(d3d4)[0] * c5c6[0]

# \partial Etotal / \partial w6
dLdw6 = -1 * h[1] * dsigmoid(d3d4)[0] * c5c6[0]

# \partial Etotal / \partial w7
dLdw7 = -1 * h[0] * dsigmoid(d3d4)[1] * c5c6[1]

# \partial Etotal / \partial w8
dLdw8 = -1 * h[1] * dsigmoid(d3d4)[1] * c5c6[1]

# \partial Etotal / \partial w1
dLdw1 = -1 * x[0] * dsigmoid(d1d2)[0] * (w5 * dsigmoid(d3d4)[0] * c5c6[0] + w7 * dsigmoid(d3d4)[1] * c5c6[1])

# \partial Etotal / \partial w2
dLdw2 = -1 * x[1] * dsigmoid(d1d2)[0] * (w5 * dsigmoid(d3d4)[0] * c5c6[0] + w7 * dsigmoid(d3d4)[1] * c5c6[1])

# \partial Etotal / \partial w3
dLdw3 = -1 * x[0] * dsigmoid(d1d2)[1] * (w6 * dsigmoid(d3d4)[0] * c5c6[0] + w8 * dsigmoid(d3d4)[1] * c5c6[1])

# \partial Etotal / \partial w4
dLdw4 = -1 * x[1] * dsigmoid(d1d2)[1] * (w6 * dsigmoid(d3d4)[0] * c5c6[0] + w8 * dsigmoid(d3d4)[1] * c5c6[1])

uw1 = w1 - alpha * dLdw1
uw2 = w2 - alpha * dLdw2
uw3 = w3 - alpha * dLdw3
uw4 = w4 - alpha * dLdw4
uw5 = w5 - alpha * dLdw5
uw6 = w6 - alpha * dLdw6
uw7 = w7 - alpha * dLdw7
uw8 = w8 - alpha * dLdw8

Etotal

0.3481129248042641

In [79]:
# intermediate values
d1d2, h, d3d4, c5c6

(array([0.775, 0.855]),
 array([0.684602, 0.701615]),
 array([1.605432, 1.744054]),
 array([-0.822776,  0.138799]))

In [80]:
# derivates
np.array([[[dLdw1, dLdw2], [dLdw3, dLdw4]], [[dLdw5, dLdw6], [dLdw7, dLdw8]]])

array([[[ 0.004284,  0.007141],
        [ 0.004459,  0.007431]],

       [[ 0.078441,  0.080391],
        [-0.012035, -0.012334]]])

In [84]:
# updated weights
np.array([[[uw1, uw2], [uw3, uw4]], [[uw5, uw6], [uw7, uw8]]])

array([[[0.497858, 0.54643 ],
        [0.597771, 0.646284]],

       [[0.660779, 0.709805],
        [0.806018, 0.856167]]])

In [85]:
# check answers for Neural Network homework using PyTorch
W1 = torch.tensor([[0.5, 0.55], [0.6, 0.65]], requires_grad=True)
W2 = torch.tensor([[0.7, 0.75], [0.8, 0.85]], requires_grad=True)
b1 = torch.tensor([0.35, 0.35])
b2 = torch.tensor([0.6, 0.6])
x = torch.tensor([0.3, 0.5])
y = torch.tensor([0.01, 0.99])

optimizer = torch.optim.SGD([W1, W2], lr=0.5)
optimizer.zero_grad()

yhat = torch.sigmoid(W2 @ torch.sigmoid(W1 @ x + b1) + b2)
E_total = 0.5*torch.sum((y - yhat)**2)
torch.set_printoptions(precision=6)
print(E_total)
E_total.backward(create_graph=True)
print("Torch gradients")
print(W1.grad)
print(W2.grad)

np.set_printoptions(precision=6)
print("Manually computed gradients")
print(np.array([[[dLdw1, dLdw2], [dLdw3, dLdw4]], [[dLdw5, dLdw6], [dLdw7, dLdw8]]]))

optimizer.step()
print("Weights after update")
print(W1)
print(W2)

print("Manually weight update")
np.array([[[uw1, uw2], [uw3, uw4]], [[uw5, uw6], [uw7, uw8]]])

tensor(0.348113, grad_fn=<MulBackward0>)
Torch gradients
tensor([[0.004284, 0.007141],
        [0.004459, 0.007431]], grad_fn=<CopyBackwards>)
tensor([[ 0.078441,  0.080391],
        [-0.012035, -0.012334]], grad_fn=<CopyBackwards>)
Manually computed gradients
[[[ 0.004284  0.007141]
  [ 0.004459  0.007431]]

 [[ 0.078441  0.080391]
  [-0.012035 -0.012334]]]
Weights after update
tensor([[0.497858, 0.546430],
        [0.597771, 0.646284]], requires_grad=True)
tensor([[0.660779, 0.709805],
        [0.806018, 0.856167]], requires_grad=True)
Manually weight update


array([[[0.497858, 0.54643 ],
        [0.597771, 0.646284]],

       [[0.660779, 0.709805],
        [0.806018, 0.856167]]])